# ZenML: Open-source MLOps Framework for reproducible ML pipelines

![Test](_assets/Logo/zenml.svg)

![Sam](_assets/sam.png)

In [ ]:
from absl import logging as absl_logging
import warnings
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2
absl_logging.set_verbosity(-10000)

Let's begin by initializing ZenML in our directory. We are going to use a local stack to begin with, for simplicity and then transition to other stacks. This can be achieved in code by executing the following block.

# Initialize ZenML

In [ ]:
!rm -rf .zen
!zenml init

## The ZenML stack

The ZenML stack is a concept that describes the union of Metadata Store, Artifact Store and Orchestrator that will be used for all pipeline runs. When you get started with zenml you start off with a default local stack.

In [ ]:
!zenml stack list

### The Local Stack

You can imagine the local stack to look like this. Within the diagram we show how a generic pipeline interacts with the local stack.

![LocalStack](_assets/localstack.png "LocalStack")

In [ ]:
!zenml stack set local_stack
# Let's train within kubeflow pipelines - this will deploy the training pipeline on a Schedule
!python run.py

We will now use the Kubeflow integration to extend the concept of stacks

![All](_assets/evidently+mlflow+discord+kubeflow.png "All")

In [ ]:
!zenml integration install kubeflow -f

### The Kubeflow Pipeline stack

Now we want to transition to a kubeflow stack that will look a little bit like this. Note that for kubeflow pipelines we also need a registry where the docker images for each step are registered. 

![KubeflowStack](_assets/localstack-with-kubeflow-orchestrator.png "KubeflowStack")

But we have good news! You barely have to do anything to transition.

In [ ]:
# You register a container registry with zenml
!zenml container-registry register local_registry --type=default --uri=localhost:5000
    
# You register an orchestrator with zenml
!zenml orchestrator register kubeflow_orchestrator --type=kubeflow

# Now it all is combined into the local_kubeflow_stack
!zenml stack register local_kubeflow_stack \
    -m local_metadata_store \
    -a local_artifact_store \
    -o kubeflow_orchestrator \
    -c local_registry

# And we activate the new stack, now all pipelines will be run within this stack
!zenml stack set local_kubeflow_stack

# Check it out, your new stack is registered
!zenml stack list

### Starting up your new kubeflow pipelines stack

All that is left to do is power up your stack. This is just one more line away. The stack up process might take some time for you. In the background k3d will be creating and starting up a cluster of docker containers to host kubeflow pipelines locally. 

In [ ]:
!k3d cluster delete zenml-kubeflow-715de2b0

In [ ]:
!zenml stack up

If you scroll down all the way on the previous output you should see a link to your running kubeflow pipelines UI. Most probably this will be at [http://localhost:8080/](http://localhost:8080/). If this does not work, you might want to try this is a seperate shell:

```
kubectl port-forward -n kubeflow svc/ml-pipeline-ui 8080:80
```

<div class="alert alert-block alert-info">
    <b>Note:</b> Currently running pipelines defined within a jupyter notebook cell is
    not supported. To get around this you can run the train pipeline within this repo. 
</div>

In [ ]:
!zenml stack set local_kubeflow_stack
# Let's train within kubeflow pipelines - this will deploy the pipeline in a one of manner
!python run.py

In [ ]:
!zenml stack set local_kubeflow_stack
# Let's train within kubeflow pipelines - this will deploy the training pipeline on a Schedule
!python run_schedule.py --interval_second 120

In [ ]:
# Do this only if the port forward from `zenml stack up` did not work. 
!kubectl port-forward -n kubeflow svc/ml-pipeline-ui 8081:80